<a href="https://colab.research.google.com/github/zhe0/prac/blob/main/pyspark_clusteringOnNumericalAndVector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install pyspark
import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession
import time

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=bc0f7e5f9e7e9a41b76e816cbdac70507f17e5f9448833b5461008f5b616f213
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [19]:
spark = SparkSession.builder.appName("ClusteringExample").getOrCreate()
spark

In [ ]:
# data = [
#     (1.0, Vectors.dense(3.0, 4.0), "a"),
#     (2.0, Vectors.dense(6.0, 7.0), "b"),
#     (3.0, Vectors.dense(9.0, 10.0), "c"),
#     (4.0, Vectors.dense(1.0, 2.0), "a"),
#     (5.0, Vectors.dense(4.0, 5.0), "b")
# ]
# df = spark.createDataFrame(data, ["feature1", "vector", "category"])
# category_indexer = StringIndexer(inputCol="category", outputCol="category_index")

#org: name	main_category	sub_category	image	link	ratings	no_of_ratings	discount_price	actual_price
#sel: name main_category sub_category ratings no_of_ratings discount_price actual_price



In [ ]:
assembler = VectorAssembler(
    inputCols=["feature1", "vector"],
    outputCol="features")

kmeans = KMeans(k=2, featuresCol="features")

pipeline = Pipeline(stages=[category_indexer, assembler, kmeans])

model = pipeline.fit(df)

predictions = model.transform(df)

In [ ]:
predictions.show()

+--------+----------+--------+--------------+--------------+----------+
|feature1|    vector|category|category_index|      features|prediction|
+--------+----------+--------+--------------+--------------+----------+
|     1.0| [3.0,4.0]|       a|           0.0| [1.0,3.0,4.0]|         1|
|     2.0| [6.0,7.0]|       b|           1.0| [2.0,6.0,7.0]|         0|
|     3.0|[9.0,10.0]|       c|           2.0|[3.0,9.0,10.0]|         0|
|     4.0| [1.0,2.0]|       a|           0.0| [4.0,1.0,2.0]|         1|
|     5.0| [4.0,5.0]|       b|           1.0| [5.0,4.0,5.0]|         1|
+--------+----------+--------+--------------+--------------+----------+



In [ ]:
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))
predictions.show()

Silhouette with squared euclidean distance = 0.6120640643838152
+--------+----------+--------+--------------+--------------+----------+
|feature1|    vector|category|category_index|      features|prediction|
+--------+----------+--------+--------------+--------------+----------+
|     1.0| [3.0,4.0]|       a|           0.0| [1.0,3.0,4.0]|         1|
|     2.0| [6.0,7.0]|       b|           1.0| [2.0,6.0,7.0]|         0|
|     3.0|[9.0,10.0]|       c|           2.0|[3.0,9.0,10.0]|         0|
|     4.0| [1.0,2.0]|       a|           0.0| [4.0,1.0,2.0]|         1|
|     5.0| [4.0,5.0]|       b|           1.0| [5.0,4.0,5.0]|         1|
+--------+----------+--------+--------------+--------------+----------+



### 寵物領養率的數據分析

* PetID - Unique hash ID of pet profile
* AdoptionSpeed - Categorical speed of adoption. Lower is faster. This is the value to predict. See below section for more info.
* Type - Type of animal (1 = Dog, 2 = Cat)
* Name - Name of pet (Empty if not named)
* Age - Age of pet when listed, in months
* Breed1 - Primary breed of pet (Refer to BreedLabels dictionary)
* Breed2 - Secondary breed of pet, if pet is of mixed breed (Refer to BreedLabels dictionary)
* Gender - Gender of pet (1 = Male, 2 = Female, 3 = Mixed, if profile represents group of pets)
* Color1 - Color 1 of pet (Refer to ColorLabels dictionary)
* Color2 - Color 2 of pet (Refer to ColorLabels dictionary)
* Color3 - Color 3 of pet (Refer to ColorLabels dictionary)
* MaturitySize - Size at maturity (1 = Small, 2 = Medium, 3 = Large, 4 = Extra Large, 0 = Not Specified)
* FurLength - Fur length (1 = Short, 2 = Medium, 3 = Long, 0 = Not Specified)
* Vaccinated - Pet has been vaccinated (1 = Yes, 2 = No, 3 = Not Sure)
* Dewormed - Pet has been dewormed (1 = Yes, 2 = No, 3 = Not Sure)
* Sterilized - Pet has been spayed / neutered (1 = Yes, 2 = No, 3 = Not Sure)
* Health - Health Condition (1 = Healthy, 2 = Minor Injury, 3 = Serious Injury, 0 = Not Specified)
* Quantity - Number of pets represented in profile
* Fee - Adoption fee (0 = Free)
* State - State location in Malaysia (Refer to StateLabels dictionary)
* RescuerID - Unique hash ID of rescuer
* VideoAmt - Total uploaded videos for this pet
* PhotoAmt - Total uploaded photos for this pet
* Description - Profile write-up for this pet. The primary language used is English, with some in Malay or Chinese.


In [ ]:
import pandas as pd

In [77]:
# download_dir = './tutorial'
# zip_file = 'https://automl-mm-bench.s3.amazonaws.com/petfinder_for_tutorial.zip'
# !wget 'https://automl-mm-bench.s3.amazonaws.com/petfinder_for_tutorial.zip'
# !unzip ./petfinder_for_tutorial.zip

# dataset_path = './petfinder_for_tutorial'
# train_data = pd.read_csv(f'{dataset_path}/train.csv', index_col=0)
# test_data = pd.read_csv(f'{dataset_path}/test.csv', index_col=0)
# label_col = 'AdoptionSpeed'
# print(len(train_data),len(test_data))
# train_data.head()

train_data = pd.read_csv('./train.csv', index_col=0)
test_data = pd.read_csv('./test.csv', index_col=0)
print(len(train_data),len(test_data))
train_data.head()

14993 3972


,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
Type,,,,,,,,,,,,,,,,,,,,,
2,Nibble,3,299,0,1,1,7,0,1,1,...,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
2,No Name Yet,1,265,0,1,1,2,0,2,2,...,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
1,Brisco,1,307,0,1,2,7,0,2,2,...,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
1,Miko,4,307,0,2,1,2,0,2,1,...,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
1,Hunter,1,307,0,1,1,0,0,2,1,...,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2


In [82]:
#remove not help columns: RescuerID, Images, PetID
print(train_data.columns)
rm=['Name','RescuerID','PetID','Images','Description','AdoptionSpeed']
tra=train_data[[i for i in train_data.columns if not i in rm]]
test=test_data[[i for i in test_data.columns if not i in rm]]
print(tra.columns)

Index(['Name', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'RescuerID',
       'VideoAmt', 'Description', 'PetID', 'PhotoAmt', 'AdoptionSpeed'],
      dtype='object')
Index(['Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2', 'Color3',
       'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed', 'Sterilized',
       'Health', 'Quantity', 'Fee', 'State', 'VideoAmt', 'PhotoAmt'],
      dtype='object')


In [83]:
# data = [
#     (1.0, Vectors.dense(3.0, 4.0), "a"),
#     (2.0, Vectors.dense(6.0, 7.0), "b"),
#     (3.0, Vectors.dense(9.0, 10.0), "c"),
#     (4.0, Vectors.dense(1.0, 2.0), "a"),
#     (5.0, Vectors.dense(4.0, 5.0), "b")
# ]


tra_df=spark.createDataFrame(list(tra.itertuples(index=False, name=None)), tra.columns.tolist())
test_df=spark.createDataFrame(list(test.itertuples(index=False, name=None)), test.columns.tolist())

cate=['Type','Breed1','Breed2','Gender','Color1','Color2','Color3','MaturitySize','FurLength','Vaccinated','Dewormed','Sterilized','Health']
category_indexer=StringIndexer(inputCols=cate, outputCols=[i+'_' for i in cate])

scale=[i for i in tra.columns.tolist() if i not in cate]
scaler =[]
for i in scale:
    scaler.append(StandardScaler(inputCol=i, outputCol=i+'_', withStd=True, withMean=False))

In [84]:
st=time.time()
assembler = VectorAssembler(inputCols=tra_df.columns,outputCol="features")

kmeans = KMeans(k=2, featuresCol="features")

pipeline = Pipeline(stages=[ category_indexer, assembler, kmeans])
model = pipeline.fit(tra_df)

predictions = model.transform(tra_df)
print('exec time: ',(time.time()-st))
predictions.show()

Py4JJavaError: An error occurred while calling o1780.fit.
: org.apache.spark.SparkException: Input column Type does not exist.
	at org.apache.spark.ml.feature.StringIndexerBase.$anonfun$validateAndTransformSchema$2(StringIndexer.scala:128)
	at scala.collection.TraversableLike.$anonfun$flatMap$1(TraversableLike.scala:293)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at scala.collection.TraversableLike.flatMap(TraversableLike.scala:293)
	at scala.collection.TraversableLike.flatMap$(TraversableLike.scala:290)
	at scala.collection.mutable.ArrayOps$ofRef.flatMap(ArrayOps.scala:198)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema(StringIndexer.scala:123)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema$(StringIndexer.scala:115)
	at org.apache.spark.ml.

to predict test dataset

In [68]:
# training dataset performance
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.7532297435812325


0.7532297435812325

In [69]:
assembler = VectorAssembler(inputCols=test_df.columns,outputCol="features")

kmeans = KMeans(k=2, featuresCol="features")

pipeline = Pipeline(stages=[ category_indexer, assembler, kmeans])
model = pipeline.fit(test_df)

predictions = model.transform(test_df)
predictions.show()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

+----+---+------+------+------+------+------+------+------------+---------+----------+--------+----------+------+--------+---+-----+--------+--------+-------------+-----+-------+-------+-------+-------+-------+-------+-------------+----------+-----------+---------+-----------+-------+--------------------+----------+
|Type|Age|Breed1|Breed2|Gender|Color1|Color2|Color3|MaturitySize|FurLength|Vaccinated|Dewormed|Sterilized|Health|Quantity|Fee|State|VideoAmt|PhotoAmt|AdoptionSpeed|Type_|Breed1_|Breed2_|Gender_|Color1_|Color2_|Color3_|MaturitySize_|FurLength_|Vaccinated_|Dewormed_|Sterilized_|Health_|            features|prediction|
+----+---+------+------+------+------+------+------+------------+---------+----------+--------+----------+------+--------+---+-----+--------+--------+-------------+-----+-------+-------+-------+-------+-------+-------+-------------+----------+-----------+---------+-----------+-------+--------------------+----------+
|   2| 24|   265|   288|     1|     1|     5| 